# **Carregar o dataset**

In [1]:
import pandas as pd # importando o pandas para manipularmos o dataset
import numpy as np # biblioteca para manipulação de vetores e matrizes grandes além de outras manipulações de dados de larga escala
import seaborn as sns # importando o Seaborn para visualizar o comportamento dos dados
import matplotlib.cm as mcm # biblioteca para mostrar gráficos (espeficamente uma parte para cores)
import matplotlib.pyplot as plt # importando o Matplotlib para o elbow method
import pandas_profiling

from pandas_profiling import ProfileReport # importando o pandas-profiling para fazer o profile do dataset
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV# utilizado para o split entre treinamento e teste
from sklearn.neighbors import KNeighborsRegressor # KNN para regressão
from sklearn.neighbors import KNeighborsClassifier # utilizado para treinar o KNN
from sklearn.linear_model import LinearRegression, LogisticRegression # Regressão linear, utilizado para treinar um modelo de classificação (regressão logística - apesar do nome é para problemas de classificação)
from sklearn.svm import SVR # SVM para regressão
from sklearn.decomposition import PCA # PCA como aprendizagem não-supervisionada
from sklearn.preprocessing import RobustScaler, OrdinalEncoder, OneHotEncoder # utilizado para que todas as entradas estejam na mesma escala numérica
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import * # importando todas as funções específicas de seleção de atributos do scikit-learn
from sklearn.decomposition import * # importando todas as funções específicas para a extração de atributos do scikit-learn
from sklearn.cluster import * # importando todas as funções específicas para o agrupamento
from sklearn.ensemble import * # importando vários ensembles para que possamos testá-los posteriormente
from sklearn.pipeline import Pipeline # utilizado para criar pipelines
from sklearn.metrics import f1_score, make_scorer # utilizado para calcular a performance dos pipelines
from lightgbm import LGBMClassifier # utilizado para treinar o LightGBM
from sklearn import set_config # utilizado para mostrar os passos do pipeline de forma visual
from sklearn.impute import SimpleImputer
from sklearn.metrics import *

set_config(display='diagram') # forçando para que os passos do pipeline sejam mostrados em visual

In [2]:
#seleção do dataset
df_nba = pd.read_csv('/content/nba_stats.csv') #carrega o dataset do diretorio 
df_nba.describe() #mostra destacaria os dados
#display(df_nba.columns)

,Ano,Idade,Jogos,TrueShootingPercentage,FreeThrowRate,OffensiveWinShares,DefensiveWinShares,WinShares,FieldGoals,FieldGoalAttempts,...,TwoPointFieldGoals,TwoPointFieldGoalAttempts,TwoPointFieldGoalPercentage,EffectiveFieldGoalPercentage,FreeThrows,FreeThrowAttempts,FreeThrowPercentage,Assists,PersonalFouls,Points
count,24624.000000,24616.000000,24624.000000,24538.000000,24525.000000,24585.000000,24585.000000,24585.000000,24624.000000,24624.000000,...,24624.000000,24624.000000,24496.000000,24525.000000,24624.000000,24624.000000,23766.000000,24624.000000,24624.000000,24624.000000
mean,1992.594989,26.664405,50.837110,0.493001,0.325455,1.257307,1.227395,2.485796,195.325820,430.645752,...,178.250447,381.756782,0.445343,0.450658,102.389336,136.775219,0.719279,114.852623,116.339222,510.116350
std,17.429594,3.841892,26.496161,0.094469,0.218971,2.136256,1.269613,3.058638,188.114361,397.624715,...,179.478923,371.260335,0.099803,0.099200,113.373565,146.078918,0.141824,135.863913,84.791873,492.922981
min,1950.000000,18.000000,1.000000,0.000000,0.000000,-5.100000,-1.000000,-2.800000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1981.000000,24.000000,27.000000,0.458000,0.208000,-0.100000,0.200000,0.200000,41.000000,99.000000,...,35.000000,82.000000,0.407000,0.414000,18.000000,27.000000,0.657000,19.000000,39.000000,106.000000
50%,1996.000000,26.000000,58.000000,0.506000,0.296000,0.400000,0.800000,1.400000,141.000000,321.000000,...,122.000000,270.000000,0.456000,0.463000,63.000000,88.000000,0.743000,68.000000,109.000000,364.000000
75%,2007.000000,29.000000,75.000000,0.544000,0.400000,1.900000,1.800000,3.800000,299.000000,661.000000,...,268.000000,579.250000,0.496000,0.501000,149.000000,201.000000,0.808000,160.000000,182.000000,778.000000
max,2017.000000,44.000000,88.000000,1.136000,6.000000,18.300000,16.000000,25.400000,1597.000000,3159.000000,...,1597.000000,3159.000000,1.000000,1.500000,840.000000,1363.000000,1.000000,1164.000000,386.000000,4029.000000


# **Divisão do dataset**

In [3]:
# criando uma cópia do dataset para fins de teste
df_nba_copy = df_nba[~pd.isnull(df_nba["WinShares"])].copy()
df_nba_copy

,Ano,Posicao,Idade,Jogos,TrueShootingPercentage,FreeThrowRate,OffensiveWinShares,DefensiveWinShares,WinShares,FieldGoals,...,TwoPointFieldGoals,TwoPointFieldGoalAttempts,TwoPointFieldGoalPercentage,EffectiveFieldGoalPercentage,FreeThrows,FreeThrowAttempts,FreeThrowPercentage,Assists,PersonalFouls,Points
0,1950,G-F,31.0,63,0.368,0.467,-0.1,3.6,3.5,144,...,144,516,0.279,0.279,170,241,0.705,176,217,458
1,1950,SG,29.0,49,0.435,0.387,1.6,0.6,2.2,102,...,102,274,0.372,0.372,75,106,0.708,109,99,279
2,1950,SF,25.0,67,0.394,0.259,0.9,2.8,3.6,174,...,174,499,0.349,0.349,90,129,0.698,140,192,438
3,1950,F,24.0,15,0.312,0.395,-0.5,-0.1,-0.6,22,...,22,86,0.256,0.256,19,34,0.559,20,29,63
4,1950,F,24.0,13,0.308,0.378,-0.5,-0.1,-0.6,21,...,21,82,0.256,0.256,17,31,0.548,20,27,59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24619,2017,PF,24.0,62,0.604,0.442,3.4,2.2,5.6,253,...,253,442,0.572,0.571,133,196,0.679,99,189,639
24620,2017,C,27.0,51,0.508,0.247,0.5,0.6,1.0,78,...,78,157,0.497,0.494,22,39,0.564,42,61,178
24621,2017,C,20.0,19,0.346,0.161,-0.1,0.1,0.0,10,...,10,31,0.323,0.323,3,5,0.600,4,17,23
24622,2017,SF,22.0,44,0.503,0.181,-0.3,0.8,0.5,88,...,55,122,0.451,0.473,31,40,0.775,36,78,240


In [4]:
# split entre treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(df_nba_copy.drop('WinShares', axis=1), # aqui informamos os atributos
                                                    df_nba_copy['WinShares'], # aqui informamos as labels e na mesma ordem dos atributos
                                                    test_size=0.25, # informamos a porcentagem de divisão da base. Geralmente é algo entre 20% (0.20) a 35% (0.35)
                                                    random_state=0) # aqui informamos um "seed". É um valor aleatório e usado para que alguns algoritmos iniciem de forma aleatória a sua divisão.

# **Pipelines sem scaler**

In [5]:
classifiers_list = [
    KNeighborsRegressor(),
    LinearRegression(),
    SVR()
]

In [6]:
for classifier in classifiers_list:
    pipe = Pipeline(steps=[('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)), ('impmean', SimpleImputer(missing_values=np.nan, strategy='mean')), ("classifier", classifier)])
    pipe.fit(X_train, y_train)
    print(classifier)
    print("Resultados do score: %.3f" % pipe.score(X_test, y_test))
    
    y_pred_sc = pipe.predict(X_test)
    display(f'Resultados de predict: {y_pred_sc}')
    display(f'Resultados real: {y_test.values}')
    display(f'R2: {r2_score(y_test, y_pred_sc)}')
    display(f'MAE: {mean_absolute_error(y_test, y_pred_sc)}')
    display(f'MSE: {mean_squared_error(y_test, y_pred_sc)}')
    display(f'RMSE: {mean_squared_error(y_test, y_pred_sc, squared=False)}')
    display(f'MAPE: {mean_absolute_percentage_error(y_test, y_pred_sc)}')


KNeighborsRegressor()
Resultados do score: 0.677


'Resultados de predict: [ 4.96  8.42 -0.18 ...  1.94  4.42  1.62]'

'Resultados real: [5.7 9.5 0.1 ... 0.5 5.  1.7]'

'R2: 0.6768621583035929'

'MAE: 0.8932942898975109'

...

'R2: 0.8708416710358469'

'MAE: 0.43965985037503996'

'MSE: 1.2050834170452376'

'RMSE: 1.097762914770415'

'MAPE: 26998725995323.01'

# **Pipelines com scaler**

In [7]:
classifiers_list = [
    KNeighborsRegressor(),
    LinearRegression(),
    SVR()
]

In [8]:
for classifier in classifiers_list:
    pipe = Pipeline(steps=[('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)), ('impmean', SimpleImputer(missing_values=np.nan, strategy='mean')), ('scaler', RobustScaler()), ("classifier", classifier)])
    pipe.fit(X_train, y_train)
    print(classifier)
    print("Resultados do score: %.3f" % pipe.score(X_test, y_test))
    
    y_pred_cs = pipe.predict(X_test)
    display(f'Resultados de predict: {y_pred_cs}')
    display(f'Resultados real: {y_test.values}')
    display(f'R2: {r2_score(y_test, y_pred_cs)}')
    display(f'MAE: {mean_absolute_error(y_test, y_pred_cs)}')
    display(f'MSE: {mean_squared_error(y_test, y_pred_cs)}')
    display(f'RMSE: {mean_squared_error(y_test, y_pred_cs, squared=False)}')
    display(f'MAPE: {mean_absolute_percentage_error(y_test, y_pred_cs)}')

KNeighborsRegressor()
Resultados do score: 0.921


'Resultados de predict: [ 5.66  9.06 -0.1  ...  1.    4.7   1.32]'

'Resultados real: [5.7 9.5 0.1 ... 0.5 5.  1.7]'

'R2: 0.9212347875498904'

'MAE: 0.419928420367659'

...

'R2: 0.9029436353549879'

'MAE: 0.18617696608425174'

'MSE: 0.905563090591404'

'RMSE: 0.9516107873450175'

'MAPE: 21919109849998.02'

#**PCA**

In [9]:
classifiers_list = [
    KNeighborsRegressor(),
    LinearRegression(),
    SVR()
]

In [10]:
for classifier in classifiers_list:
    pipe = Pipeline(steps=[('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)), ('impmean', SimpleImputer(missing_values=np.nan, strategy='mean')), ('pca', PCA(n_components=5)), ("classifier", classifier)])
    pipe.fit(X_train, y_train)
    print(classifier)
    print("Resultados do score: %.3f" % pipe.score(X_test, y_test))
    
    y_pred_pca = pipe.predict(X_test)
    display(f'Resultados de predict: {y_pred_pca}')
    display(f'Resultados real: {y_test.values}')
    display(f'R2: {r2_score(y_test, y_pred_pca)}')
    display(f'MAE: {mean_absolute_error(y_test, y_pred_pca)}')
    display(f'MSE: {mean_squared_error(y_test, y_pred_pca)}')
    display(f'RMSE: {mean_squared_error(y_test, y_pred_pca, squared=False)}')
    display(f'MAPE: {mean_absolute_percentage_error(y_test, y_pred_pca)}')
    

KNeighborsRegressor()
Resultados do score: 0.679


'Resultados de predict: [ 3.68  8.42 -0.18 ...  1.86  3.9   1.82]'

'Resultados real: [5.7 9.5 0.1 ... 0.5 5.  1.7]'

'R2: 0.6785963935750812'

'MAE: 0.9266048478932813'

...

'R2: 0.716260997305284'

'MAE: 0.8463773384933494'

'MSE: 2.6473644375753405'

'RMSE: 1.6270723516719656'

'MAPE: 44134258880382.914'

# **Conclusão**

LinearRegression(), apresentou a melhor previsão 